# Learning and benchmarking

This notebook gathers the functions needed to train agents to forage optimally, as well as tools to calculate their foraging efficiency as well as their comparison to benchmark foraging strategies.

In [ ]:
#| default_exp learn_and_bench

In [ ]:
#| export
import numpy as np
import pathlib

from rl_opts.rl_framework import TargetEnv, Forager
from rl_opts.utils import get_encounters

# Learning

In [ ]:
#|export
def learning(config, results_path, run):
    """
    Training of the RL agent
    
    Parameters
    ----------
    config : dict
        Dictionary with all the parameters
    results_path : str
        Path to save the results
    run : int
        Agent identifier
    """
    
    #Simulation parameters
    TIME_EP = config['MAX_STEP_L'] #time steps per episode
    EPISODES = config['NUM_EPISODES'] #number of episodes
    
    #initialize environment
    env = TargetEnv(Nt=config['NUM_TARGETS'], L=config['WORLD_SIZE'], r=config['r'], lc=config['lc'])
    
    #initialize agent 
    STATE_SPACE = [np.linspace(0, config['MAX_STEP_L']-1, config['NUM_BINS']), np.arange(1), np.arange(1)]
    NUM_STATES = np.prod([len(i) for i in STATE_SPACE])
    
    #default initialization policy
    if config['PI_INIT'] == 0.5:
        INITIAL_DISTR = None
    #change initialization policy
    elif config['PI_INIT'] == 0.99:
        INITIAL_DISTR = []
        for percept in range(NUM_STATES):
            INITIAL_DISTR.append([0.99, 0.01])
            
    agent = Forager(num_actions=config['NUM_ACTIONS'],
                    state_space=STATE_SPACE,
                    gamma_damping=config['GAMMA'],
                    eta_glow_damping=config['ETA_GLOW'],
                    initial_prob_distr=INITIAL_DISTR)
    
    for e in range(EPISODES):
        
        #initialize environment and agent's counter and g matrix
        env.init_env()
        agent.agent_state = 0
        agent.reset_g()
    
        for t in range(TIME_EP):
            
            #step to set counter to its min value n=1
            if t == 0 or env.kicked[0]:
                #do one step with random direction (no learning in this step)
                env.update_pos(1)
                #check boundary conditions
                env.check_bc()
                #reset counter
                agent.agent_state = 0
                #set kicked value to false again
                env.kicked[0] = 0
                
            else:
                #get perception
                state = agent.get_state()
                #decide
                action = agent.deliberate(state)
                #act (update counter)
                agent.act(action)
                
                #update positions
                env.update_pos(action)
                #check if target was found + kick if it is
                reward = env.check_encounter()
                    
                #check boundary conditions
                env.check_bc()
                #learn
                agent.learn(reward)
                
                
        if (e+1)%500 == 0:
            #save h matrix of the agent at this stage of the learning process
            np.save(results_path+'memory_agent_'+str(run)+'_episode_'+str(e+1)+'.npy', agent.h_matrix)
                

# Generate walk from a policy

In [ ]:
#|export

def walk_from_policy(policy, time_ep, n, L, Nt, r, lc, destructive=False, with_bound=False, bound=100):
    """
    Walk of foragers given a policy. Performance is evaluated as the number of targets found in a fixed time time_ep.
    
    Parameters
    ----------
    policy : list
        Starting from counter=1, prob of continuing for each counter value.
    time_ep : int
        Number of steps (decisions).
    n : int
        Number of agents that walk in parallel (all with the same policy, they do not interact). This is "number of walks" in the paper.
    L : int
        World size.
    Nt : int
        Number of targets.
    r : float
        Target radius.
    lc : float
        Cutoff length. Agent is displaced a distance lc from the target when it finds it.
    destructive : bool, optional
        True if targets are destructive. The default is False.
    with_bound : bool, optional
        True if policy is cut. The default is False.
    bound : int, optional
        Bound of the policy (maximum value for the counter). The default is 20.

    Returns
    -------
    reward : list, len(rewards)=n
        Number of targets found by each agent in time_ep steps of d=1.

    """
    
    #initialize agents clocks, positions and directions, as well as targets in the env.
    pos = np.zeros((time_ep, n, 2)) 
    pos[0] = np.random.rand(n,2)*L
    
    current_pos = np.random.rand(n,2)*L
    
    direction = np.random.rand(n)*2*np.pi 
    internal_counter = [0]*n
    target_positions = np.random.rand(Nt,2) * L
    reward = [0]*n
    
    #cut policy
    if with_bound:
        policy[bound:] = [0] * (len(policy)-bound)
        
    for t in range(1, time_ep):   
        
        #update position
        previous_pos = np.copy(current_pos)
        current_pos[:,0] = previous_pos[:, 0] + np.cos(direction)
        current_pos[:,1] = previous_pos[:, 1] + np.sin(direction)
        
        #check reward
        encounters = get_encounters(previous_pos, current_pos, target_positions, L, r)
        
        for ag, num_encounters in enumerate(np.sum(encounters,axis=0)):
            kick = False
            
            if num_encounters > 0: 
                
                first_encounter = np.arange(len(target_positions))[encounters[:,ag]]
                
                if destructive:
                    #target is destroyed, sample position for a new target.
                    target_positions[first_encounter] = np.random.rand(2) * L
                else:
                    #----KICK----
                    # If there was encounter, we reset direction and change position of particle to (pos target + lc)
                    kick_direction = np.random.rand()*2*np.pi  
                    
                    current_pos[ag, 0] = target_positions[first_encounter, 0] + lc*np.cos(kick_direction)
                    current_pos[ag, 1] = target_positions[first_encounter, 1] + lc*np.sin(kick_direction)
                    
                    #------------
                internal_counter[ag] = 0
                reward[ag] += 1
                kick = True
                
            
            current_pos[ag] %= L
            
            if np.random.rand() > policy[internal_counter[ag]] or kick:
                internal_counter[ag] = 0
                direction[ag] = np.random.rand()*2*np.pi  
                
            else:
                internal_counter[ag] += 1
                
    return reward

# Efficiency computation

In [ ]:
#|export
from rl_opts.utils import get_config, get_policy

def agent_efficiency(results_path, config, run, num_walks, episode_interval):
    """
    Computes the agent's average search efficiency over a number of walks where the agent follows a fixed policy. 
    This is repeated with the policies at different stages of the training to analyze the evolution of its performance.
    
    Parameters
    ----------
    results_path : str
        Path to the results folder, from which to extract the agent's policies
    config : dict
        Dictionary with all the parameters. It needs to be the same configuration file as the one used to train the agent.
    run : int
        Id of the agent
    num_walks : int
        Number of (independent) walks
    episode_interval : int
        Every 'episode_interval' training episodes, the policy of the agent is taken and its performance is analyzed.
        
    """

    print('Statistics postlearning of agent', run, '\nData obtained from folder: ', results_path)
    
    
    for training_episode in [i for i in range(0, config['NUM_EPISODES'] + 1, episode_interval)]:
        
        if training_episode == 0 and config['PI_INIT'] == 0.99:
            frozen_policy = [0.99 for percept in range(config['MAX_STEP_L'])] #initial policy
            
        elif training_episode == 0 and config['PI_INIT'] == 0.5:
            frozen_policy  = [0.5 for percept in range(config['MAX_STEP_L'])] #initial policy
            
        else:
            #get policy from the stored h matrix at the given training_episode
            frozen_policy = get_policy(results_path, run, training_episode)
            
        #run the 10^4 walks (in parallel) with the same policy
        rewards = walk_from_policy(policy=frozen_policy,
                                   time_ep=config['MAX_STEP_L'],
                                   n=num_walks,
                                   L=config['WORLD_SIZE'],
                                   Nt=config['NUM_TARGETS'],
                                   r=config['r'],
                                   lc=config['lc'])
        
        #save results
        np.save(results_path+'performance_post_training_agent_'+str(run)+'_episode_'+str(training_episode)+'.npy', rewards)
        
        

# Benchmarks

Code to get the search efficiency of the benchmark models. We consider Lévy and bi-exponential distributions and obtain the model parameters that achieve the highest search efficiency. We use the library `Tune` for the efficiency optimization within given parameter ranges.

In [ ]:
#| export
import pathlib

from ray import tune
from ray.tune.search.bayesopt import BayesOptSearch
from ray.tune.search import ConcurrencyLimiter

from rl_opts.analytics import get_policy_from_dist, pdf_powerlaw, pdf_multimode

In [ ]:
#| export
def average_search_efficiency(config):
    """
    Get the average search efficiency, considering the benchmark model defined in config.

    Parameters
    ----------
    config : dict
        Dictionary with the configuration of the benchmark model.
    """
    
    #get parameters of the distributions depending on the chosen model
    if config['model'] == 'powerlaw':
        #get policy from benchmark model
        policy = get_policy_from_dist(n_max = config['time_ep'], 
                                      func = pdf_powerlaw,
                                      beta = config['beta']
                                     )
    
    elif config['model'] == 'double_exp':
        #get policy from benchmark model
        policy = get_policy_from_dist(n_max=config['time_ep'],
                                      func = pdf_multimode,
                                      lambdas = np.array([config['d_int'], config['d_ext']]),
                                      probs = np.array([config['p'], 1-config['p']])
                                  )
    
    
    #run the walks in parallel
    efficiencies = walk_from_policy(policy=policy,
                                    time_ep=config['time_ep'],
                                    n=config['n'],
                                    L=config['L'],
                                    Nt=config['Nt'],
                                    r=config['r'],
                                    lc=config['lc'])
    
    #get the mean search efficiency over the walks
    mean_eff = np.mean(efficiencies) 
    tune.report(mean_eff = mean_eff)
    


### Example

Set up the configuration, run and type of search

In [ ]:
from rl_opts.learn_and_bench import average_search_efficiency
from ray import tune
from ray.tune.search.bayesopt import BayesOptSearch
from ray.tune.search import ConcurrencyLimiter
import numpy as np

In [ ]:
#### Minimal example #####
run = '0'
search_type = 'Bayesian'
config = {'num_raytune_samples': 3,
          'd_int': tune.uniform(0.00001, 20.0),
          'd_ext': 100.0,
          'p': tune.uniform(0.0, 1.0),
          'beta': None,
          'model': 'double_exp',
          'time_ep': 20,
          'n': 10,
          'lc': 3.0,
          'Nt': 100,
          'L': 100,
          'r': 0.5,
          'destructive': False,
         }

In [ ]:
#| hide
# Full simulations as in manuscript
run = '1'
search_type = 'Bayesian'
config = {'d_int': tune.uniform(0.00001, 20.0),
          'd_ext': 100.0,
          'p': tune.uniform(0.0, 1.0),
          'beta': None,
          'model': 'double_exp',
          'time_ep': 20000,
          'n': 10000,
          'lc': 3.0,
          'Nt': 100,
          'L': 100,
          'r': 0.5,
          'destructive': False
          }

Initialize `Tune`

In [ ]:
if search_type == 'Bayesian': #Bayesian optimization
    
    bayesopt = BayesOptSearch(metric="mean_eff", mode="max")
    bayesopt = ConcurrencyLimiter(bayesopt, max_concurrent=3)
    tuner = tune.Tuner(average_search_efficiency, 
                        tune_config=tune.TuneConfig(search_alg=bayesopt, num_samples=config['num_raytune_samples']), 
                        param_space=config)
    
elif search_type == 'Grid':#Grid search

    tuner = tune.Tuner(average_search_efficiency,
                        tune_config=tune.TuneConfig(num_samples=1),
                        param_space=config)

Run the algorithm

In [ ]:
result_grid = tuner.fit()

Save the results and the configuration in the corresponding folders

In [ ]:
# Results path
results_path = 'results/benchmark_models/' + config['model'] + '/'+ str(config['lc']) + '/run_'+run+'/'
pathlib.Path(results_path).mkdir(parents=True, exist_ok=True)

# Configuration path
config_path = 'configurations/benchmark_models/'
pathlib.Path(config_path).mkdir(parents=True, exist_ok=True)

# Save results
results_df = result_grid.get_dataframe()
results_df.to_csv(results_path+'df'+run+'_'+config['model']+'_lc_'+str(config['lc'])+'.csv')
np.save(config_path+'config_'+config['model']+'_lc_'+str(config['lc'])+'_run_'+run+'.npy', config)

In [ ]:
#| hide
from nbdev import nbdev_export; nbdev_export()